In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/Graduation_Project/CheXpert-v1.0-small.zip > /dev/null

In [ ]:
!pip install timm > /dev/null

In [ ]:
import cv2
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import torch 
from torch import nn
from torchsummary import summary

import timm
from torch.optim import Adam, lr_scheduler

from myGen_1 import MyGen

from sklearn.metrics import roc_auc_score

In [ ]:
train = pd.read_csv('/content/CheXpert-v1.0-small/train.csv')

In [ ]:
train.loc[:, train.columns[5:]] = train.loc[:, train.columns[5:]].fillna(0)

zeros = ['Cardiomegaly', 'Consolidation', 'Pleural Effusion']
train.loc[:, zeros] = train.loc[:, zeros].replace({-1:0})
ones = ['Edema', 'Atelectasis']
train.loc[:, ones] = train.loc[:, ones].replace({-1:1})

train = train[['Path', 'Frontal/Lateral', 'Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']].copy() 

In [ ]:
train.to_csv('/content/CheXpert-v1.0-small/train_edited.csv', index=False)

In [ ]:
save_weights_path = '/content/drive/MyDrive/Single_Weights/Template.pth' #edit save location
load_weights_path = '/content/drive/MyDrive/Final_Weights/Multi_Label/Template.pth'  #edit load location

In [ ]:
timm.list_models(pretrained=True)

In [ ]:
model = timm.create_model('mobilenetv3_large_100_miil', num_classes=5, pretrained=False) #edit model name
model = model.cuda()

summary(model, (3, 320, 320), batch_size=32, device='cuda')

model.load_state_dict(torch.load(load_weights_path))

# Test_1

In [ ]:
root = '/content/'
testGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=320,   #edit image size
                 frontal=True,
                 use_proposed_1=False,
                 use_proposed_2=False)

testloader =  torch.utils.data.DataLoader(testGen, batch_size=1, shuffle=False)


model.eval()
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for data in testloader:
        test_data, test_labels = data
        test_data = test_data.cuda()
        y_pred = model(test_data)
        y_pred = torch.sigmoid(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())
  
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)

val_auc_mean =  roc_auc_score(test_true, test_pred) 
val_auc_mean

In [ ]:
labels = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']

results = pd.DataFrame(index=labels)


scores = []
for i in range(5):
    score = roc_auc_score(test_true[:, i], test_pred[:, i])
    scores.append(score)
    
results['AUC'] = scores
results

In [ ]:
single_model = nn.Sequential(*list(model.children())[:-1], nn.Linear(1280, 1)).cuda()  #edit n units in Linear Layer

summary(single_model, (3, 320, 320), batch_size=32, device='cuda')

In [ ]:
n_params = len(list(model.parameters()))  # num of layers
idx      = 0  #edit idx according to ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']  
 
param_1 = list(model.parameters())[-2][idx,:].unsqueeze(dim=0) 
param_2 = list(model.parameters())[-1][idx].unsqueeze(dim=0)

for i, param in enumerate(single_model.parameters()):
    if i==(n_params-2):
        param.data = param_1
    elif i==(n_params-1):
        param.data = param_2

# Test_2

In [ ]:
root = '/content/'
testGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=320,  #edit image size
                 frontal=True,
                 use_proposed_1=False,
                 use_proposed_2=False,
                 col_to_train=['Cardiomegaly'])  #edit disease name

testloader =  torch.utils.data.DataLoader(testGen, batch_size=1, shuffle=False)

In [ ]:
single_model.eval()
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for data in testloader:
        test_data, test_labels = data
        test_data = test_data.cuda()
        y_pred = single_model(test_data)
        y_pred = torch.sigmoid(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())
  
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)

In [ ]:
val_auc_mean =  roc_auc_score(test_true, test_pred) 

In [ ]:
val_auc_mean

# Let's begin training

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
root = '/content/'

trainGen = MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/train_edited.csv', 
                 image_size=320, #edit
                 frontal=True,
                 use_proposed_1=True,
                 use_proposed_2=False,
                 col_to_train=['Cardiomegaly'])  #edit

valGen =  MyGen(image_root_path=root, 
                 csv_path='CheXpert-v1.0-small/valid.csv', 
                 image_size=320, #edit
                 frontal=True,
                 use_proposed_1=True,
                 use_proposed_2=False,
                 col_to_train=['Cardiomegaly'])  #edit

trainloader =  torch.utils.data.DataLoader(trainGen, batch_size=32, shuffle=True) # batchsize = 32 >> 320 if not then 64
testloader =  torch.utils.data.DataLoader(valGen, batch_size=32, shuffle=False)   # batchsize = 32 >> 320 if not then 64

# define loss & optimizer
Loss = nn.BCELoss()

optimizer = Adam(single_model.parameters(), lr=1e-3)

scheduler = lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.5)  # if batchsize = 32 >> stepsize = 8 if not >> 4 

# training
const_threshold = 8  #if batchsize = 32 >> const_threshold = 8 if not >> 4 

val_auc_mean = 0
best_val_auc = 0  
const_cnt = 0

for epoch in range(5):

    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = single_model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = Loss(y_pred, train_labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 400 == 0:
         single_model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for data in testloader:
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = single_model(test_data)
                  y_pred = torch.sigmoid(y_pred)
                  test_pred.append(y_pred.cpu().detach().numpy())            
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 
              single_model.train()

              if best_val_auc < val_auc_mean:
                  const_cnt = 0
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.5) #if batchsize = 32 >> stepsize = 8 if not >> 4 
                  best_val_auc = val_auc_mean
                  torch.save(single_model.state_dict(), save_weights_path)
              else:
                  const_cnt += 1
                  scheduler.step()            

              if (const_cnt > const_threshold):
                  const_cnt = 0 
                  scheduler = lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.5) #if batchsize = 32 >> stepsize = 8 if not >> 4 
                  single_model.load_state_dict(torch.load(save_weights_path))
                               
              lr = get_lr(optimizer)

              print('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f, lr=%f' %(epoch, idx, val_auc_mean, best_val_auc, lr))